In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
import pyreader
import numpy as np
import csv
from batchmake import Batcher
import pprint
tf.set_random_seed(777)
from tensorflow.python.layers import core

In [2]:
datas= 'data_samples/'
vocabulary = 'vocab.csv'
vocabs = []
with open(vocabulary, 'r', newline='', encoding='utf-8') as vocab:
    words = csv.reader(vocab)
    for i, word in enumerate(words):
        vocabs.append(word[0])
#         vocabs[word[0]]=i
#         vocabs[word] = i
print(len(vocabs))
word_to_id = {word: i for i, word in enumerate(vocabs)}
id_to_word = {i:word for i, word in enumerate(vocabs)}

22673


In [3]:
id_to_word[0], word_to_id['<PAD>']

('<PAD>', 0)

In [4]:
hidden_size=32
sequence=5
embedding_dim=50
attention_size = 50
batch_size=16
vocab_size=len(vocabs)
checkpoint = './seq2seq_test.ckpt'

In [5]:
def source_to_seq(text):
    '''Prepare the text for the model'''
    sequence_length = 5
    return [word_to_id.get(word, word_to_id['<UNK>']) for word in text]+ [word_to_id['<PAD>']]*(sequence_length-len(text))

In [42]:
input_sentence = 'import'
text = source_to_seq([input_sentence])
print(text)

[12, 0, 0, 0, 0]


In [54]:
text = [38,455,19,3138,0]
print(text)
text = [8,0,0,0,0]

[38, 455, 19, 3138, 0]


In [55]:
checkpoint = './seq2seq_test/seq2seq_test.ckpt'
loaded_graph = tf.Graph()

with tf.Session(graph=loaded_graph) as Ses:
    loader = tf.train.import_meta_graph(checkpoint +'.meta')
    loader.restore(Ses, checkpoint)
    
    print('loaded_graph.get_all_collection_keys()',loaded_graph.get_all_collection_keys())
    input_data = loaded_graph.get_tensor_by_name('inputs_xdata:0')
    logits_ = loaded_graph.get_tensor_by_name('predictions_sample_id:0')
    logits = loaded_graph.get_tensor_by_name('predictions_output:0')
    
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')

    
    print(logits_)
    logits = tf.nn.top_k(logits[0],5)
    answer_logits = Ses.run([logits,logits_], {input_data: [text]*batch_size, 
                                      target_sequence_length: [len(text)]*batch_size, 
                                      source_sequence_length: [len(text)]*batch_size})
    
    
pad = word_to_id['<PAD>']

print('Original Text:', input_sentence)

print('\nSource')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([id_to_word[i] for i in text])))
print(answer_logits)

INFO:tensorflow:Restoring parameters from ./seq2seq_test/seq2seq_test.ckpt
loaded_graph.get_all_collection_keys() ['trainable_variables', 'model_variables', 'variables', 'while_context', 'cond_context', 'train_op', 'queue_runners', 'summaries']
Tensor("predictions_sample_id:0", shape=(16, ?), dtype=int32)
Original Text: import

Source
  Word Ids:    [8, 0, 0, 0, 0]
  Input Words: return <PAD> <PAD> <PAD> <PAD>
[TopKV2(values=array([[ 3.81096935,  2.66198564,  2.66166663,  2.63152862,  2.29902101],
       [ 3.61593962,  2.00625348,  1.89728355,  1.65010238,  1.38964272]], dtype=float32), indices=array([[ 95,  58,   3,   4,   2],
       [  3, 402,  30,  95,   1]])), array([[95,  3],
       [95,  3],
       [95,  3],
       [95,  3],
       [95,  3],
       [95,  3],
       [95,  3],
       [95,  3],
       [95,  3],
       [95,  3],
       [95,  3],
       [95,  3],
       [95,  3],
       [95,  3],
       [95,  3],
       [95,  3]])]


In [56]:
# answer_logits[0].shape
print(answer_logits[0][1])
print('\nTarget')
print('  Word Ids:       {}'.format([i for i in answer_logits[0][1][0] if i != pad]))
print('  Response Words: {}'.format(" ".join([id_to_word[i] for i in answer_logits[0][1][0] if i != pad])))

[[ 95  58   3   4   2]
 [  3 402  30  95   1]]

Target
  Word Ids:       [95, 58, 3, 4, 2]
  Response Words: 3 a <EOS> ' <GO>
